In [ ]:
!pip install scrapy 
!pip install datefinder
!pip install wikipedia

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize
import urllib3
import csv
#from scrapy import Selector
from pandas.io.html import read_html
from scrapy import Selector
import re
import datefinder
import spacy
from sklearn.utils import shuffle
sp = spacy.load("en_core_web_sm")


In [ ]:
urls = ['https://en.wikipedia.org/wiki/Nikola_Tesla',
        'https://en.wikipedia.org/wiki/Ahmed_Zewail',
        'https://en.wikipedia.org/wiki/Ali_Moustafa_Mosharafa',
        'https://en.wikipedia.org/wiki/Isaac_Newton',
        'https://en.wikipedia.org/wiki/Albert_Einstein',
        'https://en.wikipedia.org/wiki/Thomas_Edison',
        'https://en.wikipedia.org/wiki/Mohamed_Salah',
        'https://en.wikipedia.org/wiki/Stephen_Hawking']


In [ ]:
def extract_infobox(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    tbl = soup.find("table", {"class": "infobox"})
    page_name = re.findall(r'/(\w+)', url)[-1].replace("_", " ") # extract page name from url
    if not tbl:
        return {}
    # fetch the rows of infobox 
    list_of_table_rows = tbl.findAll('tr')
    info_dic = {}
    for tr in list_of_table_rows:
        th = tr.find("th") # table header
        td = tr.find("td") # table data  
        if th is not None and td is not None:
            innerText = ''
            for i, elem in enumerate(td.recursiveChildGenerator()):
                if isinstance(elem, str):
                    clean = re.sub("([\[]).*?([\]])", "\g<1>\g<2>", elem.strip())
                    innerText += clean.replace('[]','') + ' '
                elif elem.name == 'br':
                    innerText += '\n'
            info_dic[th.text] = innerText
    # print(info)        
    return info_dic, page_name



def clean_lists(info_dict , page_name): 
    dic = {}
    for key in info_dict:
      value = info_dict[key] 
      value = value.replace("  ", ",")# replace double space with ','  
      value = value.replace("\n", ",") # replace new lines with ','
      value = value.split(",")
      value = ', '.join(value)
      value= re.sub('[!#&/;=@[\]^_`{|}~]', "", value) # remove any special characters
      # ignore Unicode non-printing space
      value = value.encode("ascii", "ignore").decode()
      key = key.encode("ascii", "ignore").decode()
      value = value.split(",")
      dic[key] = value


    clean_info = []
    for key in dic:
      clean_info.append((page_name, key, dic[key]))
    return clean_info




def make_tripls(list_of_tuples):
  sturture_info = []
  for i in list_of_tuples:
    for j in range(len(i[2])):
      sub = i[0]
      rel = ''
      obj = ''

      if (j ==0) and (i[1] =="Born"):# extract date of birth
        matches = datefinder.find_dates(i[2][0])
        date = [match for match in matches][0].strftime("%m/%d/%Y")
        rel , obj = "date of birth", date

      # extract place of birth  
      elif (j > 0) and (i[1] =="Born"):
        rel = "place of death"
        obj = ",".join(i[2][1:])

      elif (j ==0) and (i[1] =="Died"):# extract date of Death
        matches = datefinder.find_dates(i[2][0])
        date = [match for match in matches][0].strftime("%m/%d/%Y")
        rel , obj = "date of death", date
        
      # extract place of death 
      elif (j > 0) and (i[1] =="Died"):
        rel = "place of death"
        obj = ",".join(i[2][1:])

      elif i[1] == 'Citizenship': # use NER to extract countries 
        t  = ''.join(i[2])
        text = sp(t)
        results = []
        for word in text.ents:
          if word.label_ == "NORP" or word.label_ == 'GPE':
            results.append(word.text)
        rel = "country of citizenship"
        obj = ', '.join(results)

      elif i[1]== "Knownfor":# rename the relation
        rel  = "known for" 
        obj = i[2][j]

      elif i[1]== "Education":
        rel = "education"
        obj = ' ,'.join(i[2])
      
      elif i[1]== "Children":# rename the relation
        rel  = "parent of " 
        obj = i[2][j]

      else :
        rel = i[1]
        obj = i[2][j]

      info_piece = [sub,rel,obj]
      sturture_info.append(info_piece)

  triples_df = pd.DataFrame(sturture_info , columns=['subject','relation', 'object'])
  triples_df = triples_df.drop_duplicates()
  triples_df['object'].replace('', np.nan, inplace=True)
  triples_df['object'].replace(' ', np.nan, inplace=True)
  triples_df.dropna(subset = ["object"], inplace=True)
  return triples_df

In [ ]:
infobox_dict, sub_name = extract_infobox(urls[4])
infobox_dict

{'Awards': ' Barnard Medal (1920)  Nobel Prize in Physics (1921)  Matteucci Medal (1921)  ForMemRS (1921)   Copley Medal (1925)   Gold Medal of the Royal Astronomical Society (1926)   Max Planck Medal (1929)  Member of the National Academy of Sciences (1942)   Time Person of the Century (1999)  ',
 'Born': '( 1879-03-14 ) 14 March 1879 \nUlm , Kingdom of Württemberg , German Empire ',
 'Children': ' Lieserl Einstein  Hans Albert Einstein  Eduard "Tete" Einstein  ',
 'Citizenship': ' Kingdom of Württemberg , part of the German Empire (1879–1896)   Stateless (1896–1901)  Switzerland (1901–1955)  Austria , part of the Austro-Hungarian Empire (1911–1912)  Kingdom of Prussia , part of the German Empire (1914–1918)   Free State of Prussia ( Weimar Republic , 1918–1933)  United States (1940–1955)  ',
 'Died': '18 April 1955 (1955-04-18) (aged\xa076) \nPrinceton, New Jersey , U.S. ',
 'Doctoral advisor': 'Alfred Kleiner ',
 'Education': ' Federal polytechnic school in Zurich (Federal teaching 

In [ ]:
clean_dic = clean_lists(infobox_dict, sub_name)
for i in clean_dic:
  print(i)

('Albert Einstein', 'Born', ['( 1879-03-14 ) 14 March 1879 ', ' Ulm ', '  Kingdom of Wrttemberg ', '  German Empire '])
('Albert Einstein', 'Died', ['18 April 1955 (1955-04-18) (aged76) ', ' Princeton', '  New Jersey ', '  U.S. '])
('Albert Einstein', 'Citizenship', [' Kingdom of Wrttemberg ', '  part of the German Empire (18791896)', '  Stateless (18961901)', ' Switzerland (19011955)', ' Austria ', '  part of the Austro-Hungarian Empire (19111912)', ' Kingdom of Prussia ', '  part of the German Empire (19141918)', '  Free State of Prussia ( Weimar Republic ', '  19181933)', ' United States (19401955)', ' '])
('Albert Einstein', 'Education', [' Federal polytechnic school in Zurich (Federal teaching diploma', '  1900)', ' University of Zurich (PhD', '  1905)', ' '])
('Albert Einstein', 'Knownfor', [' General relativity', ' Special relativity', ' Photoelectric effect', ' Emc 2 (Massenergy equivalence)', ' Ehf (PlanckEinstein relation)', ' Theory of Brownian motion', ' Einstein field equa

In [ ]:
triplets = make_tripls(clean_dic)
triplets

,subject,relation,object
0,Albert Einstein,date of birth,03/14/1879
1,Albert Einstein,place of death,"Ulm , Kingdom of Wrttemberg , German Empire"
4,Albert Einstein,date of death,04/18/1955
5,Albert Einstein,place of death,"Princeton, New Jersey , U.S."
8,Albert Einstein,country of citizenship,"German, Switzerland, Austria, German, Weimar R..."
...,...,...,...
98,Albert Einstein,Influences,Michele Besso
99,Albert Einstein,Influences,Moritz Schlick
100,Albert Einstein,Influences,Thomas Young
101,Albert Einstein,Influences,Gottfried Wilhelm Leibniz


In [ ]:
triplets.to_csv(r'/content/example_riplet.csv')

In [ ]:
dfs = []
for url in urls:
  infobox_dict, sub_name = extract_infobox(url)
  clean_dic = clean_lists(infobox_dict, sub_name)
  triplets = make_tripls(clean_dic)
  dfs.append(triplets)

final_triples = shuffle(pd.concat(dfs))
final_triples


,subject,relation,object
5,Thomas Edison,Burial place,Thomas Edison National Historical Park
10,Ahmed Zewail,Almamater,(Ph.D.)
12,Stephen Hawking,Spouse(s),Jane Wilde
55,Albert Einstein,Awards,Gold Medal of the Royal Astronomical Society...
10,Nikola Tesla,Discipline,Electrical engineering
...,...,...,...
36,Ahmed Zewail,Doctoral advisor,Robin M. Hochstrasser
9,Thomas Edison,Occupation,businessman
33,Nikola Tesla,Significant design,Tesla's oscillator
78,Isaac Newton,Preceded by,Isaac Barrow


In [ ]:
final_triples.to_csv(r'/content/info_triplet.csv')